In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("Google Drive mounted successfully!")

Mounted at /content/drive
Google Drive mounted successfully!


In [5]:
# --- Set Up Project Directory ---
# This cell creates a dedicated folder for your project within Google Drive
# and changes the current working directory into it. All subsequent files
# will be created and managed within this folder.
import os

drive_project_path = "/content/drive/MyDrive/de_weather_project"

if not os.path.exists(drive_project_path):
    os.makedirs(drive_project_path)
    print(f"Created project directory in Drive: {drive_project_path}")
else:
    print(f"Project directory already exists in Drive: {drive_project_path}")

os.chdir(drive_project_path)

print(f"Changed current working directory to: {os.getcwd()}")

Project directory already exists in Drive: /content/drive/MyDrive/de_weather_project
Changed current working directory to: /content/drive/MyDrive/de_weather_project


In [6]:
# --- Create fetch_weather_data.py Script ---
# This cell writes the Python script responsible for the data pipeline.
# It fetches current weather data from the Open-Meteo API for multiple cities,
# processes it, and stores it into an SQLite database (weather.db).
# The script handles API requests, JSON parsing, data type conversions,
# and structured data storage with a composite primary key.
%%writefile fetch_weather_data.py
import requests
import pandas as pd
import sqlite3
import os
import datetime
import time

# --- Configuration ---
# List of representative locations (Latitude, Longitude, Timezone, Continent, City)
# You can customize this list to include cities relevant to your analysis.
LOCATIONS = [
    # North America
    {"latitude": 34.0522, "longitude": -118.2437, "timezone": "America/Los_Angeles", "continent": "North America", "city": "Los Angeles"},
    {"latitude": 40.7128, "longitude": -74.0060, "timezone": "America/New_York", "continent": "North America", "city": "New York"},
    {"latitude": 45.4215, "longitude": -75.6972, "timezone": "America/Toronto", "continent": "North America", "city": "Ottawa"},

    # South America
    {"latitude": -23.5505, "longitude": -46.6333, "timezone": "America/Sao_Paulo", "continent": "South America", "city": "Sao Paulo"},
    {"latitude": -33.4489, "longitude": -70.6693, "timezone": "America/Santiago", "continent": "South America", "city": "Santiago"},

    # Europe
    {"latitude": 51.5074, "longitude": -0.1278, "timezone": "Europe/London", "continent": "Europe", "city": "London"},
    {"latitude": 48.8566, "longitude": 2.3522, "timezone": "Europe/Paris", "continent": "Europe", "city": "Paris"},
    {"latitude": 52.5200, "longitude": 13.4050, "timezone": "Europe/Berlin", "continent": "Europe", "city": "Berlin"},

    # Asia
    {"latitude": 35.6895, "longitude": 139.6917, "timezone": "Asia/Tokyo", "continent": "Asia", "city": "Tokyo"},
    {"latitude": 28.7041, "longitude": 77.1025, "timezone": "Asia/Kolkata", "continent": "Asia", "city": "New Delhi"},
    {"latitude": 39.9042, "longitude": 116.4074, "timezone": "Asia/Shanghai", "continent": "Asia", "city": "Beijing"},

    # Africa
    {"latitude": -26.2041, "longitude": 28.0473, "timezone": "Africa/Johannesburg", "continent": "Africa", "city": "Johannesburg"},
    {"latitude": 30.0444, "longitude": 31.2357, "timezone": "Africa/Cairo", "continent": "Africa", "city": "Cairo"},

    # Australia/Oceania
    {"latitude": -33.8688, "longitude": 151.2093, "timezone": "Australia/Sydney", "continent": "Australia", "city": "Sydney"},
    {"latitude": -36.8485, "longitude": 174.7633, "timezone": "Pacific/Auckland", "continent": "Australia", "city": "Auckland"}
]

API_BASE_URL = "https://api.open-meteo.com/v1/forecast"
DB_FILE = "weather.db"

def fetch_current_weather(latitude, longitude, timezone):
    """
    Fetches current weather data from Open-Meteo API.
    Returns a dictionary of weather data or None if an error occurs.
    """
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True,
        "timezone": timezone
    }

    try:
        response = requests.get(API_BASE_URL, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()

        if 'current_weather' in data:
            return data['current_weather']
        else:
            print(f"Warning: 'current_weather' key not found in API response for {latitude}, {longitude}.")
            return None

    except requests.exceptions.HTTPError as e:
        print(f"API HTTP Error for {latitude}, {longitude}: {e.response.status_code} - {e.response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"API request general error for {latitude}, {longitude}: {e}")
        return None

def create_weather_table(conn):
    """
    Creates the weather_records table in the SQLite database if it doesn't exist,
    with a composite primary key for multiple locations.
    """
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS weather_records (
        timestamp TEXT NOT NULL,
        temperature_celsius REAL,
        windspeed_ms REAL,
        weather_code INTEGER,
        latitude REAL NOT NULL,
        longitude REAL NOT NULL,
        timezone TEXT,
        continent TEXT,
        city TEXT,
        ingestion_timestamp TEXT,
        PRIMARY KEY (timestamp, latitude, longitude)
    );
    """
    try:
        cursor = conn.cursor()
        cursor.execute(create_table_sql)
        conn.commit()
        print(f"SQLite table 'weather_records' processed successfully (created or already existed).")
    except sqlite3.Error as e:
        print(f"Error creating table: {e}")

def store_weather_data(data_frame, conn):
    """
    Stores weather data from a Pandas DataFrame into the SQLite database.
    """
    if data_frame.empty:
        return

    try:
        if 'timestamp' in data_frame.columns and pd.api.types.is_datetime64_any_dtype(data_frame['timestamp']):
            data_frame['timestamp'] = data_frame['timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')

        if 'ingestion_timestamp' in data_frame.columns and pd.api.types.is_datetime64_any_dtype(data_frame['ingestion_timestamp']):
            data_frame['ingestion_timestamp'] = data_frame['ingestion_timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f')

        data_frame.to_sql('weather_records', conn, if_exists='append', index=False)
    except sqlite3.IntegrityError:
        pass # Suppress integrity error for duplicate primary keys
    except sqlite3.Error as e:
        print(f"Error storing data to DB: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during data storage: {e}. Data: {data_frame.to_dict('records')}")


def main():
    print(f"--- Starting weather data pipeline for multiple locations ---")

    conn = None
    try:
        conn = sqlite3.connect(DB_FILE)
        create_weather_table(conn)

        records_processed = 0
        for loc in LOCATIONS:
            current_weather_data = fetch_current_weather(loc['latitude'], loc['longitude'], loc['timezone'])

            time.sleep(0.1) # Small delay to avoid hitting API rate limits

            if current_weather_data:
                current_weather_data['ingestion_timestamp'] = datetime.datetime.now(datetime.timezone.utc)
                current_weather_data['latitude'] = loc['latitude']
                current_weather_data['longitude'] = loc['longitude']
                current_weather_data['timezone'] = loc['timezone']
                current_weather_data['continent'] = loc['continent']
                current_weather_data['city'] = loc['city']

                df = pd.DataFrame([current_weather_data])

                df['timestamp'] = pd.to_datetime(df['time'], errors='coerce')
                df['temperature_celsius'] = pd.to_numeric(df['temperature'], errors='coerce')
                df['windspeed_ms'] = pd.to_numeric(df['windspeed'], errors='coerce')
                df['weather_code'] = pd.to_numeric(df['weathercode'], errors='coerce').astype('Int64')

                df_to_store = df[[
                    'timestamp', 'temperature_celsius', 'windspeed_ms',
                    'weather_code', 'latitude', 'longitude', 'timezone',
                    'continent', 'city', 'ingestion_timestamp'
                ]].copy() # Explicitly make a copy here to prevent SettingWithCopyWarning

                store_weather_data(df_to_store, conn)
                records_processed += 1
            else:
                print(f"Skipping data storage for {loc['city']} due to fetch error.")

        print(f"\n--- Data pipeline finished. Total records processed this run: {records_processed} ---")

    except Exception as e:
        print(f"An error occurred during the main pipeline execution: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

if __name__ == "__main__":
    main()

Overwriting fetch_weather_data.py


In [8]:
# --- Run Data Pipeline (Populate DB) ---
# This cell executes the `fetch_weather_data.py` script.
# It triggers the data collection process, fetching current weather data
# and storing it into your 'weather.db' database.
!python fetch_weather_data.py
print("Data collection complete.")

--- Starting weather data pipeline for multiple locations ---
SQLite table 'weather_records' processed successfully (created or already existed).

--- Data pipeline finished. Total records processed this run: 15 ---
Database connection closed.
Data collection complete.


In [9]:
# --- Verify Data in DB ---
# This cell connects to your 'weather.db' and displays a preview of the collected data.
# It also provides counts to confirm successful data ingestion and shows records per city/continent.
import sqlite3
import pandas as pd
import os

# Ensure you are in the correct directory (where weather.db is located)
drive_project_path = "/content/drive/MyDrive/de_weather_project"
os.chdir(drive_project_path)

DB_FILE = "weather.db"
conn = None
try:
    conn = sqlite3.connect(DB_FILE)
    print(f"Connected to SQLite database: {DB_FILE}")

    # Query to view a sample of records
    query_data = "SELECT timestamp, city, continent, temperature_celsius, windspeed_ms, weather_code FROM weather_records ORDER BY timestamp DESC, city ASC LIMIT 10;"
    print("\n--- Sample Records from weather_records table ---")
    df_data = pd.read_sql_query(query_data, conn)
    print(df_data)

    # Query to count the total number of records
    query_count = "SELECT COUNT(*) AS total_records FROM weather_records;"
    print("\n--- Total Number of Records in weather_records table ---")
    df_count = pd.read_sql_query(query_count, conn)
    print(df_count)

    # Query to count records per city/continent
    query_city_counts = """
    SELECT
        continent,
        city,
        COUNT(*) AS record_count
    FROM
        weather_records
    GROUP BY
        continent, city
    ORDER BY
        continent, city;
    """
    print("\n--- Records Count per City/Continent ---")
    df_city_counts = pd.read_sql_query(query_city_counts, conn)
    print(df_city_counts)

except sqlite3.Error as e:
    print(f"Error accessing SQLite database '{DB_FILE}': {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    if conn:
        conn.close()
        print("\nDatabase connection closed.")

Connected to SQLite database: weather.db

--- Sample Records from weather_records table ---
             timestamp          city  continent  temperature_celsius  \
0  2025-07-18T10:30:00      Auckland  Australia                 12.1   
1  2025-07-18T08:30:00        Sydney  Australia                  6.2   
2  2025-07-18T07:30:00         Tokyo       Asia                 27.6   
3  2025-07-18T06:30:00       Beijing       Asia                 25.5   
4  2025-07-18T04:00:00     New Delhi       Asia                 27.8   
5  2025-07-18T01:30:00         Cairo     Africa                 28.9   
6  2025-07-18T00:30:00        Berlin     Europe                 17.9   
7  2025-07-18T00:30:00  Johannesburg     Africa                  9.0   
8  2025-07-18T00:30:00         Paris     Europe                 23.0   
9  2025-07-17T23:30:00        London     Europe                 21.9   

   windspeed_ms  weather_code  
0          15.0             3  
1           2.3             2  
2           6.3    

In [12]:
# --- Set Up Git & Push Project to GitHub ---
# This cell initializes your Git repository, configures your identity,
# commits all project files (script, database, and this notebook),
# and pushes them to your GitHub repository.
#
# IMPORTANT: Ensure your Personal Access Token (PAT) is correctly saved
# in Colab Secrets under the name `GH_TOKEN` for this to work securely.
from google.colab import userdata

# Ensure you are in your project directory
drive_project_path = "/content/drive/MyDrive/de_weather_project"
os.chdir(drive_project_path)

print("--- Setting up Git and Pushing to GitHub ---")

# Initialize Git repository (creates .git folder or re-initializes if present)
# Removing .git first ensures a completely fresh Git state for a clean push.
!rm -rf .git
!git init
print("Git repository initialized (or re-initialized).")

# Configure Git user identity (replace with your GitHub username and email)
!git config --global user.name "TrangDao0630"
!git config --global user.email "thutrangtt00@gmail.com"
print("Git user configuration set.")

# Add all project files to the staging area
# This includes fetch_weather_data.py, weather.db, and data_pipeline_notebook.ipynb
!git add .
print("All project files added to staging.")

# Create the first commit
!git commit -m "Initial commit: weather data pipeline and colab notebook"
print("New first commit created.")

# Retrieve PAT from Colab Secrets
github_token = None
try:
    github_token = userdata.get('GH_TOKEN')
    if not github_token:
        raise ValueError("GitHub PAT (GH_TOKEN) not found in Colab Secrets or not enabled for this notebook.")
    print("Successfully retrieved GitHub PAT from Colab Secrets.")
except Exception as e:
    print(f"Error accessing Colab Secret: {e}")


# Set remote URL using the securely retrieved PAT
# This defines where your 'origin' remote points to for pushing/pulling
repo_url_with_pat = f"https://{github_token}@github.com/TrangDao0630/de_weather_project.git"
# Add the remote. '|| true' prevents error if 'origin' was somehow already added.
!git remote add origin https://github.com/TrangDao0630/de_weather_project.git || true
# Then, set the URL with PAT for authentication.
!git remote set-url origin {repo_url_with_pat}
print("Git remote URL set for authentication.")

# Force push to overwrite the remote history. This is necessary for a clean start.
!git push --force origin master
print("\n--- Push to GitHub complete! Check your repository: https://github.com/TrangDao0630/de_weather_project ---")

--- Setting up Git and Pushing to GitHub ---
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/de_weather_project/.git/
Git repository initialized (or re-initialized).
Git user configuration set.
All project files added to staging.
[master (root-commit) bc59a32] Initial commit: weather data pipeline and colab notebook
 3 files changed, 175 insertions(+)
 create mode 100644 data_pipeline_notebook.ipynb
 create mode 100644 fetch_weather_data.py
 create mode 100644 weather.db
New first commit created.
Su

In [13]:
# --- Data Exploration (SQL Queries) ---
# This section demonstrates how to use SQL queries to analyze the collected weather data.
# It helps extract meaningful insights and understand trends from the 'weather.db' database.
import sqlite3
import pandas as pd
import os

drive_project_path = "/content/drive/MyDrive/de_weather_project"
os.chdir(drive_project_path)

DB_FILE = "weather.db"
conn = None
try:
    conn = sqlite3.connect(DB_FILE)
    print(f"Connected to SQLite database: {DB_FILE}")

    # SQL Query 1: Overall Average Temperature and Wind Speed by Continent
    print("\n--- Overall Average Weather by Continent ---")
    query_1 = """
    SELECT
        continent,
        COUNT(DISTINCT city) AS num_cities,
        COUNT(*) AS total_records,
        AVG(temperature_celsius) AS avg_temperature_celsius,
        AVG(windspeed_ms) AS avg_windspeed_ms
    FROM
        weather_records
    GROUP BY
        continent
    ORDER BY
        continent;
    """
    df_avg_continent = pd.read_sql_query(query_1, conn)
    print(df_avg_continent)

    # SQL Query 2: Hottest and Coldest Cities (from collected data)
    print("\n--- Hottest and Coldest Cities (based on collected data) ---")
    query_2 = """
    SELECT
        city,
        continent,
        MAX(temperature_celsius) AS max_temperature_celsius,
        MIN(temperature_celsius) AS min_temperature_celsius,
        AVG(temperature_celsius) AS avg_temperature_celsius
    FROM
        weather_records
    GROUP BY
        city, continent
    ORDER BY
        avg_temperature_celsius DESC;
    """
    df_temp_extremes = pd.read_sql_query(query_2, conn)
    print(df_temp_extremes)

    # SQL Query 3: Most Common Weather Codes per Continent
    print("\n--- Most Common Weather Codes per Continent ---")
    query_3 = """
    SELECT
        continent,
        weather_code,
        COUNT(*) AS count_of_observations
    FROM
        weather_records
    GROUP BY
        continent, weather_code
    ORDER BY
        continent, count_of_observations DESC;
    """
    df_weather_codes = pd.read_sql_query(query_3, conn)
    print(df_weather_codes)

    # SQL Query 4: Hourly/Daily Temperature Trend for a Specific City
    # Note: For more comprehensive trends, run the data pipeline (Cell 4)
    # multiple times over a period (e.g., hours or days) to collect historical data.
    print("\n--- Hourly/Daily Temperature Trend for a Specific City (e.g., New York) ---")
    query_4 = """
    SELECT
        strftime('%Y-%m-%d %H:00:00', timestamp) AS hour_of_day,
        city,
        AVG(temperature_celsius) AS avg_temperature_celsius,
        AVG(windspeed_ms) AS avg_windspeed_ms
    FROM
        weather_records
    WHERE
        city = 'New York' -- Change this city to see trends for other locations
    GROUP BY
        hour_of_day, city
    ORDER BY
        hour_of_day ASC;
    """
    df_city_trend = pd.read_sql_query(query_4, conn)
    print(df_city_trend)

except sqlite3.Error as e:
    print(f"Error accessing SQLite database '{DB_FILE}': {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    if conn:
        conn.close()
        print("\nDatabase connection closed.")

Connected to SQLite database: weather.db

--- Overall Average Weather by Continent ---
       continent  num_cities  total_records  avg_temperature_celsius  \
0         Africa           2              4                19.075000   
1           Asia           3              6                26.883333   
2      Australia           2              4                 8.125000   
3         Europe           3              6                19.966667   
4  North America           3              6                27.933333   
5  South America           2              4                15.150000   

   avg_windspeed_ms  
0          4.500000  
1          5.766667  
2          8.500000  
3          5.766667  
4         17.966667  
5          2.925000  

--- Hottest and Coldest Cities (based on collected data) ---
            city      continent  max_temperature_celsius  \
0       New York  North America                     32.5   
1          Cairo         Africa                     29.3   
2          T

In [14]:
import os

# Ensure you are in your project directory
drive_project_path = "/content/drive/MyDrive/de_weather_project"
os.chdir(drive_project_path)
print(f"Current working directory: {os.getcwd()}")

# --- Add, Commit, and Push the Updated Notebook ---
print("\n--- Adding, committing, and pushing updated notebook (with SQL exploration) to GitHub ---")

# Add the notebook file to staging (Git will detect changes since last commit)
# Use 'data_pipeline_notebook.ipynb' or '.' if you want to add all changed files
!git add data_pipeline_notebook.ipynb
print("Updated notebook added to staging.")

# Commit the changes with the requested message
!git commit -m "Explore data using SQL queries: Add SQL analysis and outputs to notebook"
print("Changes committed.")

# Push the changes to GitHub
# Assuming your Git remote is correctly set up (from your last successful push)
!git push origin master
print("Push command executed. Check GitHub to see your updated notebook!")

Current working directory: /content/drive/MyDrive/de_weather_project

--- Adding, committing, and pushing updated notebook (with SQL exploration) to GitHub ---
Updated notebook added to staging.
[master 2511c76] Explore data using SQL queries: Add SQL analysis and outputs to notebook
 1 file changed, 1 insertion(+), 1 deletion(-)
Changes committed.
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 2.34 KiB | 266.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/TrangDao0630/de_weather_project.git
   bc59a32..2511c76  master -> master
Push command executed. Check GitHub to see your updated notebook!
